<h1 align="center"> TRABALHO - REST API - GRUPO 3</h1>

<h2> INTRODUÇÃO </h2>

<p> O objetivo do trabalho é de filtrar notícias disponíveis por uma API de acordo com um tema, salvando os resultados em uma base de dados/arquivo, tratar esses dados para a geração de relatórios e disponibilizar a consulta dos produtos gerados através de uma API utilizando o Flask. </p>
<p>A base deve ser atualizada com requisições realizadas a cada hora na fonte dos artigos: https://newsapi.org/ </p>

<h2> Metodologia </h2>

<p> O trabalho proposto estruturou sua execução da seguinte forma:

1. um script ("requisicao.py") para chamada da API 'newsapi' com os parâmetros de estudo, construindo a base de dados com arquivo .csv e atualizando
o mesmo sempre que um novo artigo for encontrado, periodicamente;

2. um script ("selects.py") para a criação dos 'relatórios', que serão consultados na API de consulta ao público;

3. um script ("flask_api.py") para manter ativa uma REST API, com os endpoints de consulta dos dados gerais, dados pelo ID e relatórios;

</p>

<p> O trecho abaixo corresponde ao script de consulta da API e criação/atualização da base de dados. Foram criadas duas funções principais para isso.</p>


In [ ]:
import pandas as pd
import requests
from datetime import datetime


apiKey = "df97b6ec3d3148c2a0f34e552e3f53af"
keyWords = "bitcoin+blockchain+web3"
today = datetime.now().strftime('%Y-%m-%d')

def busca_tema():

    url = f'https://newsapi.org/v2/everything?q={keyWords}&from=2024-03-10&to={today}&language=en&sortBy=publishedAt&page=1&apiKey={apiKey}'
    response = requests.get(url)

    if response.status_code == 200:

      results = response.json()
      print("Requisição feita com sucesso!.")
    else:
      return print("Problema na requisição.")

    return results


def atualizar_csv():
    df_novo = pd.json_normalize(busca_tema()['articles'])

    try:
        df_antigo = pd.read_csv('code.csv', index_col=0)
        df_atualizado = pd.concat([df_antigo, df_novo]).drop_duplicates(subset=['publishedAt', 'author', 'title'], keep='last')
        df_atualizado.sort_values('publishedAt').reset_index(drop=True).to_csv('code.csv', index=True)
        print(f"Arquivo CSV atualizado com sucesso em {datetime.now()}.")

    except FileNotFoundError:
        df_novo.sort_values('publishedAt').reset_index(drop=True).to_csv('code.csv', index=True)
        print(f"Arquivo CSV criado com sucesso em {datetime.now()}.")

atualizar_csv()

# Para atualização a cada 1 hora basta colocar a ultima linha do codigo dentro de um while true: 

# While True:
#   atualizar_csv()
#   time.sleep(3600)  # 3600 segundos equivalente a 1 hora, assim o arquivo csv atualizará de 1 em 1 hora


<h3> Script dos relatórios (consultas) </h3>

<p> Para atender as demandas e possibilitar responder as perguntas dos usuários, os dados foram manipulados com pandas, permitindo a geraçao de relatórios diversos,
os quais foram armazenados em arquivos .csv </p>

O fluxograma para atingir o objetivo foi:
1. consultar a base dos dados;
2. manipular através da importação com pandas e utilizando o método 'groupby';
3. armazenar  através de '.csv' para disponibilizar posteriormente na API;

In [ ]:
import re
import pandas as pd
import requests
from datetime import datetime

df_historico = pd.read_csv("code.csv", index_col = 0)



# trasformacao;
df_historico['data'] = df_historico.publishedAt.apply(lambda x: x[:10])


# 4.1 - Quantidade de notícias por ano, mês e dia de publicação;
noticias_por_data = df_historico.groupby(['data'], as_index=False).count()[['data', 'title']]
noticias_por_data.columns = ['data', 'n_artigos']
noticias_por_data


#     - 4.2 - Quantidade de notícias por fonte e autor;
noticias_por_fonte_autor = df_historico.groupby(['source.name', 'author'], as_index=False).size()#.reset_index(name='Quantidade')
noticias_por_fonte_autor


# 4.3 - Quantidade de aparições das 3 palavras-chave por ano, mês e dia de publicação definidas no item 2
palavras_chave = ['bitcoin', 'blockchain', 'web3']
for palavra in palavras_chave:
    df_historico[palavra] = df_historico['title'].str.count(palavra, flags=re.IGNORECASE)

aparicoes_palavras_chave = df_historico.groupby(['data'], as_index = False)[palavras_chave].sum()
aparicoes_palavras_chave


# Salvando as transformações em arquivo csv
noticias_por_data.to_csv("noticias_por_data.csv",sep=",", index = False)
noticias_por_fonte_autor.to_csv("noticias_por_fonte_autor.csv",sep=",",index=False)
aparicoes_palavras_chave.to_csv("aparicoes_palavras_chave.csv",sep=",",index=False)


<p> Para permitir a consulta da base criada, foi utilizado a biblioteca Flask, que permite inicializar uma REST API simples e funcional.</p>
<p> Os diferentes endpoints da API direcionam para os arquivos salvos no projeto, permitindo a leitura dos dados gerais, dados por ID
(através do método 'iloc' do pandas) e os diferentes relatórios no escopo do projeto ('flask_api.py'). </p>
<p> A API permite que o usuário submeta um artigo caso deseje e que não exista na base de dados, utilizando 'POST' ('post.py'). </p>

In [ ]:
# flask_api.py

import pandas as pd
import requests
#import pyspark.pandas as ps 
from flask import Flask, jsonify, request, make_response


# app.config["JSON_SORT_KEYS"] = False

app = Flask(
    __name__
)  # instância o método Flask com o nome do app igual ao nome do arquivo.py



dados = pd.read_csv('./code.csv')
noticias_por_data = pd.read_csv('./noticias_por_data.csv')
noticias_por_fonte = pd.read_csv('./noticias_por_fonte_autor.csv')
aparicoes_keyWords = pd.read_csv('./aparicoes_palavras_chave.csv')

@app.route("/", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def homepage():

    return jsonify({"mensagem": "olá, você está na página principal", "pagina": 1})

@app.route("/dados", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def todos_dados():

    
    return jsonify(
        dados.to_json())

@app.route("/dados/<int:id>", methods = ["GET"])
def retorna_id(id):

    try:
        df_id = dados.loc[[id]].to_json()
        return jsonify(
            df_id)
    except:
        print("id não encontrado")
        return jsonify(
            dados.to_json())

@app.route("/dados/noticias_data", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def noticias_dada():

    
    return jsonify(
        noticias_por_data.to_json())

@app.route("/dados/noticias_fonte", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def noticias_fonte():

    
    return jsonify(
        noticias_por_fonte.to_json())

@app.route("/dados/aparicoes_palavraschaves", methods=["GET"])  # define o endpoint da página e qual método ela aceita
def aparicoes_chaves():

    
    return jsonify(
        aparicoes_keyWords.to_json())


@app.route("/dados", methods = ["POST"])
def recebe_noticia():

    novo_artigo = request.get_json()
    df_novoArtigo = pd.DataFrame(novo_artigo, index = [0])

    df = pd.read_csv('code.csv', index_col=0)

    df = pd.concat([df, df_novoArtigo]
            ).reset_index(drop = True)
    

    if df.duplicated(subset=['title', 'author']).sum() >=0:
        print("Artigo ja existente no banco de dados!")

        return jsonify(novo_artigo)

    # export.
    else:
        df.to_csv('code.csv')

        return jsonify(df.to_json())

##

if __name__ == "__main__":
    app.run(port = 5001, debug=True)

In [ ]:
# post.py

import requests
import pandas as pd

#inp == dados inputados pelo usuario para o POST;
inp = {'author': 'Test1', 'title': 'Test', 'description': 'more Test',
       'url': 'www.Test.com.br', 'urlToImage': 'htpps://Test', 
       'publishedAt': '2024-04-07 T19:00:00Z'}

url = 'http://127.0.0.1:5001/dados'

requests.post(url, json = inp)

In [ ]:
# schedule with CRON?

<h2> Resultados </h2>


#### Nossa Api Flask inicializada:

- Página inicial:

![Homepage](./prints/Captura_homepage.png)

- Pagina com os dados gerais:

![Dados](./prints/Captura_dados.png)

- Pagina com ID (ID = 50):

![Dados_Id](./prints/Captura_dadosID.png)

- Pagina com os notícias por ano, mês e dia de publicação:

![Noticias_data](./prints/Captura_noticias_data.png)

- Pagina com os notícias por fonte e autor:

![Noticias_fonte](./prints/Captura_noticias_fonte.png)

- Pagina com aparições das 3 palavras-chave:

![Aparicoes_palavraschave](./prints/Captura_aparicoes_palavraschaves.png)

## Consumindo nossa API

In [1]:
import pandas as pd
import requests

#### Pagina com os Dados Gerais

In [2]:
# PAGINA COM OS DADOS GERAIS
urlDados = "http://127.0.0.1:5001/dados"

response = requests.get(urlDados)

if response.status_code == 200:

  results = response.json()
  print("Requisição feita com sucesso!.")
else:
  print("Problema na requisição.")

results

Requisição feita com sucesso!.


'{"Unnamed: 0":{"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"11":11,"12":12,"13":13,"14":14,"15":15,"16":16,"17":17,"18":18,"19":19,"20":20,"21":21,"22":22,"23":23,"24":24,"25":25,"26":26,"27":27,"28":28,"29":29,"30":30,"31":31,"32":32,"33":33,"34":34,"35":35,"36":36,"37":37,"38":38,"39":39,"40":40,"41":41,"42":42,"43":43,"44":44,"45":45,"46":46,"47":47,"48":48,"49":49,"50":50,"51":51,"52":52,"53":53,"54":54,"55":55,"56":56,"57":57,"58":58,"59":59,"60":60,"61":61,"62":62,"63":63,"64":64,"65":65,"66":66,"67":67,"68":68,"69":69,"70":70,"71":71,"72":72,"73":73,"74":74,"75":75,"76":76,"77":77,"78":78,"79":79,"80":80,"81":81,"82":82,"83":83,"84":84,"85":85,"86":86,"87":87,"88":88,"89":89,"90":90,"91":91,"92":92,"93":93,"94":94,"95":95,"96":96,"97":97,"98":98,"99":99},"author":{"0":null,"1":"Daily Contributors","2":"Danny Nelson","3":"Nick Dunn","4":"InvestorsObserver","5":"Hassan Maishera","6":"AMBCrypto Team","7":"Lyllah Ledesma, Omkar Godbole","8":"Particle Network

In [3]:
df_dados = pd.read_json(results)

df_dados.head()

C:\Users\Lucas Guimarães\AppData\Local\Temp\ipykernel_17224\3837113771.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_dados = pd.read_json(results)


,Unnamed: 0,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,0,None,[Removed],[Removed],https://removed.com,None,1970-01-01T00:00:00Z,[Removed],None,[Removed]
1,1,Daily Contributors,Owning Versus Renting – The Circumstances of W...,"By Daily Contributors\nLast week, Charles Dray...",https://www.hackread.com/owning-versus-renting...,https://www.hackread.com/wp-content/uploads/20...,2024-03-19T17:50:05Z,"Last week, Charles Dray from Resonance Securit...",None,HackRead
2,2,Danny Nelson,Polygon Labs Paid $4M to Host Starbucks' Doome...,The blockchain developer gave the coffee giant...,https://www.coindesk.com/business/2024/03/19/p...,https://www.coindesk.com/resizer/l363QHdllMJoP...,2024-03-19T17:59:30Z,<ul><li>Polygon Labs paid $4 million to host S...,None,CoinDesk
3,3,Nick Dunn,Top Crypto Gainers on 19 March – SCS and NULLS,The crypto market capitalization at large has ...,https://techreport.com/crypto-news/top-crypto-...,https://techreport.com/wp-content/uploads/2024...,2024-03-20T01:33:41Z,The crypto market capitalization at large has ...,None,Techreport.com
4,4,InvestorsObserver,Flash News: Merlin Chain Now Supported on OKX ...,Flash News: Merlin Chain Now Supported on OKX ...,https://www.investorsobserver.com/news/qm-pr/4...,https://s3.amazonaws.com/images.investorsobser...,2024-03-20T03:42:00Z,"SINGAPORE\r\n,\r\nMarch 20, 2024\r\n/PRNewswir...",None,InvestorsObserver


#### Pagina com nosso ID

In [4]:
url_ID = "http://127.0.0.1:5001/dados/50"

response_id = requests.get(url_ID)

if response_id.status_code == 200:

  results_id = response_id.json()
  print("Requisição feita com sucesso!.")
else:
  print("Problema na requisição.")

results_id

Requisição feita com sucesso!.


'{"Unnamed: 0":{"50":50},"author":{"50":"Hassan Maishera"},"title":{"50":"Dogecoin rallies by 28% this week: Will other memecoins follow suit?"},"description":{"50":"Key takeaways Dogecoin is the best performer amongst the top 10 cryptocurrencies by market cap this week. Bitcoin Dogs prepares to carry out the biggest NFT mint on Bitcoin following its presale. DOGE tops $0.20 after this week\\u2019s rally DOGE, the native coin o\\u2026"},"url":{"50":"https:\\/\\/coinjournal.net\\/news\\/dogecoin-rallies-by-28-this-week-will-other-memecoins-follow-suit\\/"},"urlToImage":{"50":"https:\\/\\/coinjournal.net\\/wp-content\\/uploads\\/2023\\/07\\/1690278872522-632a3e80-c55b-4247-a34a-a60ef4e194a8.jpg"},"publishedAt":{"50":"2024-03-30T11:07:00Z"},"content":{"50":"Key takeaways\\r\\n<ul><li>Dogecoin is the best performer amongst the top 10 cryptocurrencies by market cap this week.\\r\\n<\\/li><li>Bitcoin Dogs prepares to carry out the biggest NFT mint on Bitcoin followi\\u2026 [+3081 chars]"},"s

In [5]:
df_id = pd.read_json(results_id)

df_id

C:\Users\Lucas Guimarães\AppData\Local\Temp\ipykernel_17224\2543407245.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_id = pd.read_json(results_id)


,Unnamed: 0,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
50,50,Hassan Maishera,Dogecoin rallies by 28% this week: Will other ...,Key takeaways Dogecoin is the best performer a...,https://coinjournal.net/news/dogecoin-rallies-...,https://coinjournal.net/wp-content/uploads/202...,2024-03-30T11:07:00Z,Key takeaways\r\n<ul><li>Dogecoin is the best ...,NaN,Coinjournal.net


#### Pagina noticias por data

In [6]:
url_noticias_data = "http://127.0.0.1:5001/dados/noticias_data"

response_noticias_data = requests.get(url_noticias_data)

if response_noticias_data.status_code == 200:

  results_noticias_data = response_noticias_data.json()
  print("Requisição feita com sucesso!.")
else:
  print("Problema na requisição.")

results_noticias_data

Requisição feita com sucesso!.


'{"data":{"0":"1970-01-01","1":"2024-03-19","2":"2024-03-20","3":"2024-03-21","4":"2024-03-22","5":"2024-03-23","6":"2024-03-24","7":"2024-03-25","8":"2024-03-26","9":"2024-03-27","10":"2024-03-28","11":"2024-03-29","12":"2024-03-30","13":"2024-03-31","14":"2024-04-01","15":"2024-04-02","16":"2024-04-03","17":"2024-04-04","18":"2024-04-05","19":"2024-04-07"},"n_artigos":{"0":1,"1":2,"2":8,"3":11,"4":8,"5":2,"6":3,"7":4,"8":2,"9":1,"10":4,"11":4,"12":1,"13":2,"14":4,"15":15,"16":12,"17":8,"18":7,"19":1}}'

In [7]:
df_noticias_data = pd.read_json(results_noticias_data)

df_noticias_data


C:\Users\Lucas Guimarães\AppData\Local\Temp\ipykernel_17224\1059015633.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_noticias_data = pd.read_json(results_noticias_data)


,data,n_artigos
0,1970-01-01,1
1,2024-03-19,2
2,2024-03-20,8
3,2024-03-21,11
4,2024-03-22,8
5,2024-03-23,2
6,2024-03-24,3
7,2024-03-25,4
8,2024-03-26,2
9,2024-03-27,1


#### Pagina noticias por fonte e autor

In [9]:
url_noticias_fonte = "http://127.0.0.1:5001/dados/noticias_fonte"

response_noticias_fonte = requests.get(url_noticias_fonte)

if response_noticias_fonte.status_code == 200:

  results_noticias_fonte = response_noticias_fonte.json()
  print("Requisição feita com sucesso!.")
else:
  print("Problema na requisição.")

results_noticias_fonte 

Requisição feita com sucesso!.


'{"source.name":{"0":"ARTnews","1":"Ambcrypto.com","2":"Biztoc.com","3":"Biztoc.com","4":"Biztoc.com","5":"Biztoc.com","6":"Biztoc.com","7":"Biztoc.com","8":"Biztoc.com","9":"Biztoc.com","10":"Biztoc.com","11":"CoinDesk","12":"CoinDesk","13":"CoinDesk","14":"CoinDesk","15":"Coinjournal.net","16":"Coinjournal.net","17":"Coinjournal.net","18":"CryptoSlate","19":"CryptoSlate","20":"CryptoSlate","21":"CryptoSlate","22":"Cryptocurrencyjobs.co","23":"Cryptocynews.com","24":"Decrypt","25":"Decrypt","26":"Dothewoo.io","27":"Financial Post","28":"Finextra","29":"Finovate.com","30":"Forbes","31":"Forbes","32":"Forbes","33":"Forbes","34":"Forbes","35":"Forbes","36":"Forbes","37":"Gadgets360.com","38":"GlobeNewswire","39":"GlobeNewswire","40":"GlobeNewswire","41":"GlobeNewswire","42":"GlobeNewswire","43":"HackRead","44":"International Business Times","45":"Investing.com","46":"Investing.com","47":"Investing.com","48":"InvestorsObserver","49":"NerdWallet","50":"Project Syndicate","51":"SiliconANGLE

In [10]:
df_noticias_fonte = pd.read_json(results_noticias_fonte)

df_noticias_fonte

C:\Users\Lucas Guimarães\AppData\Local\Temp\ipykernel_17224\1421451268.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_noticias_fonte = pd.read_json(results_noticias_fonte)


,source.name,author,size
0,ARTnews,Harrison Jacobs,1
1,Ambcrypto.com,AMBCrypto Team,5
2,Biztoc.com,ankr.com,1
3,Biztoc.com,beincrypto.com,1
4,Biztoc.com,coindesk.com,1
...,...,...,...
57,TheBlaze,Jon Stokes,1
58,VentureBeat,Dean Takahashi,1
59,ZyCrypto,PR DESK,2
60,newsBTC,Jake Simmons,2


#### Pagina Aparições das 3 palavras chave

In [11]:
url_aparicoes_palavraschaves = "http://127.0.0.1:5001/dados/aparicoes_palavraschaves"

response_aparicoes_palavraschaves = requests.get(url_aparicoes_palavraschaves)

if response_aparicoes_palavraschaves.status_code == 200:

  results_aparicoes_palavraschaves = response_aparicoes_palavraschaves.json()
  print("Requisição feita com sucesso!.")
else:
  print("Problema na requisição.")

results_aparicoes_palavraschaves 

Requisição feita com sucesso!.


'{"data":{"0":"1970-01-01","1":"2024-03-19","2":"2024-03-20","3":"2024-03-21","4":"2024-03-22","5":"2024-03-23","6":"2024-03-24","7":"2024-03-25","8":"2024-03-26","9":"2024-03-27","10":"2024-03-28","11":"2024-03-29","12":"2024-03-30","13":"2024-03-31","14":"2024-04-01","15":"2024-04-02","16":"2024-04-03","17":"2024-04-04","18":"2024-04-05","19":"2024-04-07"},"bitcoin":{"0":0,"1":0,"2":2,"3":1,"4":3,"5":1,"6":1,"7":0,"8":0,"9":0,"10":3,"11":0,"12":0,"13":1,"14":1,"15":8,"16":4,"17":1,"18":2,"19":0},"blockchain":{"0":0,"1":0,"2":1,"3":0,"4":1,"5":1,"6":0,"7":0,"8":1,"9":0,"10":0,"11":0,"12":0,"13":1,"14":0,"15":1,"16":2,"17":5,"18":1,"19":0},"web3":{"0":0,"1":1,"2":0,"3":2,"4":1,"5":1,"6":0,"7":0,"8":0,"9":0,"10":1,"11":0,"12":0,"13":0,"14":2,"15":6,"16":1,"17":0,"18":1,"19":0}}'

In [12]:
df_aparicoes_palavraschaves = pd.read_json(results_aparicoes_palavraschaves)

df_aparicoes_palavraschaves

C:\Users\Lucas Guimarães\AppData\Local\Temp\ipykernel_17224\2738398616.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_aparicoes_palavraschaves = pd.read_json(results_aparicoes_palavraschaves)


,data,bitcoin,blockchain,web3
0,1970-01-01,0,0,0
1,2024-03-19,0,0,1
2,2024-03-20,2,1,0
3,2024-03-21,1,0,2
4,2024-03-22,3,1,1
5,2024-03-23,1,1,1
6,2024-03-24,1,0,0
7,2024-03-25,0,0,0
8,2024-03-26,0,1,0
9,2024-03-27,0,0,0
